## Erzeugung von Testdaten mittels *Ragas*

In dieser Übung geht es darum, mittels [Ragas](https://docs.ragas.io/) automatisch Testfragen zu den Wahlprogrammen der Parteien zu erstellen. 
Dabei wollen wir zwei Aspekte anpassen:

- Wir wollen konkrete *Personas* für die Testragen vorgeben,
- die Fragen sollen auf Deutsch erstellt werden.

In [1]:
!pip install  --upgrade langchain langchain-community langchain_chroma langchain_openai langchain_unstructured  unstructured[pdf] chromadb ragas

  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.14-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_unstructured-0.1.6-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.2.10-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached onnxruntime-1.19.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached unstructured_client-0.28.1-py3-none-any.whl.metadata (20 kB)
  Using cached chardet-5.2.

In [2]:
import chromadb
import os
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_unstructured.document_loaders import UnstructuredLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements
from unstructured.documents.elements import Image

Wir verwenden wieder die `.env` Datei aus dem letzten Praktikum.

In [3]:
load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#RAGAS_APP_TOKEN = os.getenv("RAGAS_APP_TOKEN")
DATABASE_PATH = "./chroma/"
EMBEDDING_MODEL = "text-embedding-ada-002"

**Bitte ändern Sie das voreingestellte Modell *nicht*!!**

In [4]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

### Laden der Parteiprogramme

Wir laden zunächst die Dateien. Prüfen Sie anschließend, ob alle Dokumente geladen wurden.

In [5]:
from langchain_community.document_loaders import DirectoryLoader

path = "./data/"
loader = DirectoryLoader(path, glob="**/*.pdf")
docs = loader.load()

### Generieren eines Testdatensatzes mit Default-Einstellungen

In [6]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/7 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/7 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/7 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/565 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/1057 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

### Bewertung des Testsets

Wandeln Sie das Testset in ein Pandas Dataframe und schauen Sie sich die Fragen an.

- Wie viele sind auf Deutsch?
- Welche Haltung/Einstellung würden Sie bei einem menschlichen Fragesteller jeweils vermuten?
- Sind die Fragen "neutral" oder kommt die Haltung einer Partei zum Ausdruck? 

In [8]:
dir(dataset)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_is_protocol',
 'cost_cb',
 'features',
 'from_annotated',
 'from_dict',
 'from_hf_dataset',
 'from_jsonl',
 'from_list',
 'from_pandas',
 'get_sample_type',
 'run_id',
 'samples',
 'to_csv',
 'to_evaluation_dataset',
 'to_hf_dataset',
 'to_jsonl',
 'to_list',
 'to_pandas',
 'total_cost',
 'total_tokens',
 'upload',
 'validate_samples']

In [10]:
df = dataset.to_pandas()

In [17]:
df['user_input'][3]

'Wie kann die Pharmaindustrie in Deutschland reformiert werden, um soziale Gerechtigkeit zu fördern?'

### Customizing der Testset-Generierung

Fügen Sie weitere *Personas* hinzu, die unterschiedliche Bevölkerungsgruppen und politische Ansichten wiederspiegeln.

In [ ]:
from ragas.testset.persona import Persona

personas = [
    Persona(
        name="Dorfbewohner aus einem ländlichen Gebiet",
        role_description="Ein Bürger aus einem ländlichen Gebiet, der sich um die Infrastruktur auf dem Land sorgt und im Alltag auf den Individualverkehr angewiesen ist.",
    ),
]

In [21]:
generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas
)

In [22]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

Schauen Sie sich die erzeugte `distribution` an. Was fällt Ihnen auf?

Wir passen nun die Sprache der Testerzeugung an.

In [24]:
for query, _ in distribution:
    prompts = await query.adapt_prompts("deutsch", llm=generator_llm)
    query.set_prompts(**prompts)

Was hat sich geändert?

In [ ]:
dataset = generator.generate_with_langchain_docs(
    docs,
    testset_size=5,
    query_distribution=distribution,
)

Vergleichen Sie das neu erzeugte Testset:

- Inwiefern spiegeln sich die Personas in den Fragen wieder?
- Wie viele Fragen sind auf Deutsch?
- Vergleichen Sie Ihre Personas und Testsets mit denen Ihrer Kommilitonen. Wie beurteilen Sie die Qualität der erstellten Testfragen?  